# BERT on IMDB Sentiment Analysis

这个文件在IMDB上进行情感分类，熟悉transformers库的使用。本notebook中不再使用torchtext。测试以下模型的表现：

* BERT(FT)

目前的进度：

* 未完成

问题：

参考：

## Requirement

## Import

In [1]:
import torch
import numpy as np
import random
from torch import nn,optim
from sklearn import metrics
import torch.nn.functional as F
from datasets import list_datasets, load_dataset, list_metrics, load_metric

use_cuda=torch.cuda.is_available()
device=torch.device("cuda" if use_cuda else "cpu")

SEED = 1234
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
if use_cuda:
    torch.cuda.manual_seed(SEED)

## 可供调整的参数

In [2]:
bs=128
d_hidden=256
d_output=2
n_layers=2
bidirectional=True
dropout=0.25
model_name='1'
max_epochs=1
require_improvement=3

## Data

In [30]:
from transformers import AutoTokenizer
imdb_dataset = load_dataset('imdb')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
imdb_dataset=imdb_dataset.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length'), batched=True)
imdb_dataset.set_format(type='torch', columns=['input_ids','attention_mask', 'label'])
test_data=imdb_dataset['test']
trainandval=imdb_dataset['train'].train_test_split(0.2)
train_data=trainandval['train']
val_data=trainandval['test']
trainloader=torch.utils.data.DataLoader(train_data,batch_size=bs)
testloader=torch.utils.data.DataLoader(test_data,batch_size=bs)
valloader=torch.utils.data.DataLoader(val_data,batch_size=bs)

Reusing dataset imdb (C:\Users\hp\.cache\huggingface\datasets\imdb\plain_text\1.0.0\90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3)
Loading cached processed dataset at C:\Users\hp\.cache\huggingface\datasets\imdb\plain_text\1.0.0\90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3\cache-9cbc6789f27de315.arrow
Loading cached processed dataset at C:\Users\hp\.cache\huggingface\datasets\imdb\plain_text\1.0.0\90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3\cache-55f3559756763399.arrow
Loading cached processed dataset at C:\Users\hp\.cache\huggingface\datasets\imdb\plain_text\1.0.0\90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3\cache-0cbf39e1281169c3.arrow
Loading cached split indices for dataset at C:\Users\hp\.cache\huggingface\datasets\imdb\plain_text\1.0.0\90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3\cache-5f55952d4785a1d3.arrow and C:\Users\hp\.cache\huggingface\datasets\imdb\plain_text\1.0.0\90099cb4

TypeError: __init__() got an unexpected keyword argument 'device'

In [29]:
next(iter(trainloader))['label'].dtype

torch.int64

## Model

In [32]:
from transformers import BertForSequenceClassification,AdamW
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)
model

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [27]:
with torch.no_grad():
    for batch in trainloader:
        x=batch['input_ids']
        m=batch['attention_mask']
        y=batch['label']
        if use_cuda:
            x.to(device)
            model.to(device)
            y.to(device)
            m.to(device)
            criterion.to(device)
        output=model(x,attention_mask=m, labels=y)
        print(output.shape)
        print(criterion(output,y))
    

KeyboardInterrupt: 

测试一下能否跑通

## Training

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [31]:
def train(model, train_iter, dev_iter, test_iter):
    if use_cuda:
        model.to(device)
    model.train()
    #optimizer = optim.Adam(model.parameters())
    optimizer = AdamW(model.parameters(), lr=5e-5)
    #criterion = nn.BCEWithLogitsLoss()

    # 学习率指数衰减，每次epoch：学习率 = gamma * 学习率
    # scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
    dev_best_loss = float('inf')
    last_improve = 0  # 记录上次验证集loss下降的batch数
    #writer = SummaryWriter(log_dir=config.log_path + '/' + time.strftime('%m-%d.%H.%M', time.localtime())+'_'+which_data+'_'+which_model+'_'+which_task+'_'+exp_number)
    
    for epoch in range(max_epochs):
        start_time = time.time()
        train_loss=0
        train_correct=0
        # scheduler.step() # 学习率衰减
        for i, batch in enumerate(train_iter):
            optimizer.zero_grad()
            x=batch['input_ids']
            m=batch['attention_mask']
            y=batch['label']
            if use_cuda:
                x.to(device)
                y.to(device)
                m.to(device)
            loss,outputs = model(x,attention_mask=m, labels=y)
            #loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            #训练集的准确率
            preds = torch.max(outputs, 1)[1]
            #preds = torch.round(torch.sigmoid(outputs))
            train_correct+=(y==preds).sum()
            train_loss+=loss.item()
        train_loss/=len(train_iterator)   #train_loss
        train_acc=train_correct/len(train_iterator.dataset)   #train_acc
            
        #验证集
        dev_acc, dev_loss = evaluate(model, dev_iter)
        end_time = time.time()
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        if dev_loss < dev_best_loss:
            dev_best_loss = dev_loss
            improve = '*'
            last_improve=epoch
            torch.save(model.state_dict(),model_name+'.pth')
        else:
            improve = ''
        msg = 'Epoch: {0:>3},  Epoch Time: {1}m {2}s,  Train Loss: {3:>5.2},  Train Acc: {4:>6.2%},  Val Loss: {5:>5.2},  Val Acc: {6:>6.2%} {7}'
        print(msg.format(epoch+1,epoch_mins, epoch_secs,train_loss, train_acc, dev_loss, dev_acc, improve))
        #writer.add_scalar("loss/train", loss.item(), total_batch)
        #writer.add_scalar("loss/dev", dev_loss, total_batch)
        #writer.add_scalar("acc/train", train_acc, total_batch)
        #writer.add_scalar("acc/dev", dev_acc, total_batch)

        if epoch - last_improve > require_improvement:
            # 验证集loss超过1epoch没下降，结束训练
            print("No optimization for a long time, auto-stopping...")
            break
    #writer.close()
    #训练跑完了，使用最佳模型测试
    model.load_state_dict(torch.load(model_name+'.pth'))
    test(model, test_iter)

def evaluate(model, data_iter, test=False):
    model.eval()
    loss_total = 0
    predict_all = np.array([], dtype=int)
    labels_all = np.array([], dtype=int)
    with torch.no_grad():
        for batch in data_iter:
            x=batch['input_ids']
            m=batch['attention_mask']
            y=batch['label']
            if use_cuda:
                x.to(device)
                y.to(device)
                m.to(device)
            loss,outputs = model(x,attention_mask=m, labels=y)
            #loss = F.cross_entropy(outputs, y)
            #loss=criterion(outputs,labels)
            loss_total += loss.item()
            y = y.data.cpu().numpy()
            predic = torch.max(outputs, 1)[1].cpu().numpy()
            #predic=torch.round(torch.sigmoid(outputs)).cpu().numpy()
            labels_all = np.append(labels_all, y)
            predict_all = np.append(predict_all, predic)
    model.train()
    acc = metrics.accuracy_score(labels_all, predict_all)
    
    if test:
        report = metrics.classification_report(labels_all, predict_all, labels=[0,1],target_names=['pos','neg'], digits=4,output_dict=True)
        confusion = metrics.confusion_matrix(labels_all, predict_all)
        return acc, loss_total / len(data_iter), report, confusion
    
    return acc, loss_total / len(data_iter)


def test(model, test_iter):
    test_acc, test_loss, test_report, test_confusion = evaluate(model, test_iter, test=True)
    msg = 'Test Loss: {0:>5.2},  Test Acc: {1:>6.2%}'
    print(msg.format(test_loss, test_acc))
    print("Precision, Recall and F1-Score...")
    print(test_report)
    print("Confusion Matrix...")
    print(test_confusion)

In [ ]:
#original
train(model,trainloader,valloader,testloader)